<a href="https://colab.research.google.com/github/JaredLevi18/Stock-market-predictor./blob/main/model_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install alpha_vantage -q  # Python module to get stock data/cryptocurrencies from the Alpha Vantage API

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from alpha_vantage.timeseries import TimeSeries

In [ ]:
config = {
    "alpha_vantage":{
        "key": "the api from alpha vintage",
        "symbol": "IBM",
        "outputsize": "full",
        "key_adjustedclose": "5. adjusted close",
    },
    "data": {
        "window_size": 20,
        "train_split_size": 0.80,
    },
    "plots": {
        "show_plots": True,
        "xticks_interval": 90,
        "color_actual": "#001f3f",
        "color_train": "#3D9970",
        "color_val": "#0074D9",
        "color_pred_train": "#3D9970",
        "color_pred_val": "#0074D9",
        "color_pred_test": "#FF4136",
    },
    "model":{
        "input_size": 1, # only 1 because we're using the closing price
        "num_lstm_layers": 2,
        "lstm_size": 32,
        "dropout": 0.2,
    },
    "training": {
        "device": "cuda",
        "batch_size": 64,
        "num_epochs": 100,
        "learning_rate": 0.01,
        "scheduler_step_size": 40,
    },
}

In [ ]:

def download_data(config, plot=False):
  ts = TimeSeries(key=config["alpha_vantage"]["key"])
  data, meta_data = ts.get_daily_adjusted(config["alpha_vantage"]["symbol"], outputsize=config["alpha_vantage"]["outputsize"])

  data_date = [date for data in data.keys()]
  data_date.reverse()

  data_close_price = [float(data[date][config["alpha_vantage"]["key_adjusted_close"]]) for date in data.keys()]
  data_close_price.reverse()
  data_close_price = np.array(data_close_price)

  num_data_points = len(data_date)
  display_date_range = "from" + data.date[0] + "to" + data_date[num_data_points - 1]
  print("Number of data points", num_data_points, display_date_range)

  if plot:
    fig = figure(figsize=(25,5), dpi=180)
    fig.patch.set_facecolor((1.0, 1.0, 1.0))
    plt.plot(data_date, data_close_price, color=config["plots"]["color_actual"])
    xticks = [data_date[i] if ((%iconfig["plots"]["xticks_interval"] == 0 and (num_data_points-i) > config["plots"]["xticks_interval"]) or i == num_data_points-1) else None for i in range(num_data_points)]
    x = np.arange(0,len(xticks))
    plt.xticks(x, xtricks, rotation='vertical')
    plt.title("Daily close price for" + config["alpha_vantage"]["symbol"] + ", " + display_date_range)
    plt.grid(b=None, which="major", axis="y", linestyle="--")
    plt.show()

  return data_date, data_close_price, num_data_points, display_date_range

data_date, data_close_price, num_data_points, display_date_range = download_data(config, plot=config["plots"]["show_plots"])

In [ ]:
# normalizing raw financial data
class Normalizer():
  def __init__(self):
    self.mu = None
    self.sd = None

  def fit_transform(self, x):
    self.mu = np.mean(x, axis=(0), keepdims=True)
    self.sd = np.std(x, axis=(0), keepdims=True)
    normalized_x = (x - self.mu) / self.sd
    return normilized_x

  def inverse_transform(self, x):
    return (x * self.sd) + self.mu

# normilze
scaler = Normalizer()
normalized_data_close_price = scaler.fit_transform(data_close_price)

In [ ]:
# preparing data: generating val and traing datasets
def prepare_x_y(x, window_size):